In [12]:
import pandas as pd
import numpy as np
import os as os 
from IPython.display import display

### Load data

In [2]:
dfs_train = {}
dfs_test = {}

In [51]:
for root, directory, files in os.walk("data/train_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_train[file[:-4]] = pd.read_csv(f"{root}/{file}")
for root, directory, files in os.walk("data/test_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_test[file[:-4]] = pd.read_csv(f"{root}/{file}")

In [50]:
def impute_nan_values(dfs):
    for df in dfs.values():
        for col in df.columns:
            if df[col].dtype == np.float64 or df[col].dtype == np.int64:
                df[col] = df.groupby("season")[col].transform(lambda x: x.fillna(x.mean()))

impute_nan_values(dfs_train)
impute_nan_values(dfs_test)

In [62]:
# show all rows with at least one nan value 
for df in dfs_train.values():
    display(df["Avg_bookie_prediction"].isna().sum()/df.shape[0]*100)
# display(dfs_train["df_belgium"][dfs_train["df_belgium"].isna().any(axis=1)])

0.3095411507647487

0.021776476566091607

0.03938041480703597

0.015942606616181746

1.0922330097087378

...

0.03231539828728389

0.19157088122605362

0.04582651391162029

0.15685818857840006

0.12549019607843137